In [1]:
!pip install transformers
import transformers
import torch

# Load the pre-trained BERT model
model_name = 'bert-base-uncased'
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)
model = transformers.BertForSequenceClassification.from_pretrained(model_name)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [2]:
%cd /content/drive/MyDrive/
import pandas as pd
# Load the dataset from a CSV file
df = pd.read_csv('labeled_twcs.csv')

/content/drive/MyDrive


<ipython-input-2-a6d364293691>:4: DtypeWarning: Columns (1,2,3,4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('labeled_twcs.csv')


In [3]:
from sklearn.utils import resample
# Separate positive and negative examples
df_pos = df[df['label'] == 'POSITIVE']
df_neg = df[df['label'] == 'NEGATIVE']

# Downsample negative and positive examples to match each other
df_neg_downsampled = resample(df_neg, replace=False, n_samples=100000, random_state=42)
df_pos_downsampled = resample(df_pos, replace=False, n_samples=100000, random_state=42)

# Combine the positive and negative examples
df= pd.concat([df_pos_downsampled, df_neg_downsampled])

In [4]:
from tqdm.auto import tqdm
# Define the maximum sequence length
max_len = 512
df = df.fillna("")

# Tokenize the text and convert to input IDs and attention masks
input_ids = []
attention_masks = []
# Create a progress bar
pbar = tqdm(total=len(df))

for text in df['text']:
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        truncation = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )        
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    # Update the progress bar for each batch
    pbar.update()

# Close the progress bar
pbar.close()
# Convert the lists to tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


  0%|          | 0/200000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
import itertools
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])
labels = torch.tensor(list(df['label_encoded']))

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=42, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=42, test_size=0.2)

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 4
max_split_size_mb = 256

# Training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size,
                               pin_memory=True, num_workers=0)

# Validation set
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size,
                                    pin_memory=True, num_workers=0)


In [12]:
from torch import nn
import numpy as np

def flat_accuracy(preds, labels):
    """
    Computes the accuracy of the predictions.
    """
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def loss_fn(logits, labels):
    """
    Computes the cross-entropy loss.
    """
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(logits.view(-1, 2), labels.view(-1))
    return loss

In [7]:
# Fine-tune the pre-trained BERT model on the training set
from transformers import AdamW, get_linear_schedule_with_warmup

# Set the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

# Set the number of epochs to train for
epochs = 4

# Start training loop
for epoch in range(epochs):
    # Set initial loss value for each epoch
    train_loss = 0
    # Iterate over the batches in the training set
    for step, batch in enumerate(train_dataloader):
        # Step 1: Load batch data onto GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out any previously calculated gradients
        model.zero_grad()
        # Step 2: Perform a forward pass
        outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        loss = outputs[0]
        # Step 3: Accumulate the training loss over all of the batches so far
        train_loss += loss.item()
        # Step 4: Perform a backward pass to calculate the gradients
        loss.backward()
        # Step 5: Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Step 6: Update parameters
        optimizer.step()
        scheduler.step()
        # Print training progress every 100 steps
        if step % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{step}/{len(train_dataloader)}], Loss: {loss.item():.4f}")
    # Calculate the average loss over all of the batches
    avg_train_loss = train_loss / len(train_dataloader)            
    print("Epoch {} of {}:".format(epoch+1, epochs))
    print("  Training loss: {:.5f}".format(avg_train_loss))
    
    # Evaluate the model on the validation set
    # Put the model in evaluation mode
    model.eval()
    # Tracking variables 
    eval_accuracy, eval_loss = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Step 1: Load batch data onto GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Step 2: Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Step 3: Perform a forward pass
            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)
            logits = outputs[0]
        # Step 4: Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Step 5: Calculate the accuracy and loss for this batch
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        eval_loss += loss_fn(outputs[0], b_labels).item()
        nb_eval_steps += 1

    # Calculate the average accuracy and loss over all of the batches
    avg_eval_accuracy = eval_accuracy / nb_eval_steps
    avg_eval_loss = eval_loss / len(validation_dataloader)
    print("  Validation loss: {:.5f}".format(avg_eval_loss))
    print("  Validation Accuracy: {:.5f}".format(avg_eval_accuracy))

    # Save the model checkpoint if the validation loss is the best we've seen so far
    if avg_eval_loss < best_eval_loss:
        best_eval_loss = avg_eval_loss
        torch.save(model.state_dict(), 'best_model_state.bin')
        print("  Best model saved")

    # Put the model back in training mode
    model.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch [1/4], Step [0/40000], Loss: 0.7515
Epoch [1/4], Step [100/40000], Loss: 0.2011
Epoch [1/4], Step [200/40000], Loss: 0.0670
Epoch [1/4], Step [300/40000], Loss: 2.0771
Epoch [1/4], Step [400/40000], Loss: 0.0474
Epoch [1/4], Step [500/40000], Loss: 0.9978
Epoch [1/4], Step [600/40000], Loss: 2.7049
Epoch [1/4], Step [700/40000], Loss: 0.0035
Epoch [1/4], Step [800/40000], Loss: 0.0107
Epoch [1/4], Step [900/40000], Loss: 0.0169
Epoch [1/4], Step [1000/40000], Loss: 0.4768
Epoch [1/4], Step [1100/40000], Loss: 0.0117
Epoch [1/4], Step [1200/40000], Loss: 0.0040
Epoch [1/4], Step [1300/40000], Loss: 1.8387
Epoch [1/4], Step [1400/40000], Loss: 0.0032
Epoch [1/4], Step [1500/40000], Loss: 0.1204
Epoch [1/4], Step [1600/40000], Loss: 0.0728
Epoch [1/4], Step [1700/40000], Loss: 1.1803
Epoch [1/4], Step [1800/40000], Loss: 1.2713
Epoch [1/4], Step [1900/40000], Loss: 0.9733
Epoch [1/4], Step [2000/40000], Loss: 1.0179
Epoch [1/4], Step [2100/40000], Loss: 1.3914
Epoch [1/4], Step [220

KeyboardInterrupt: ignored

In [13]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_bert_model(model, dataloader):
    # Put the model in evaluation mode
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Tracking variables 
    eval_accuracy, eval_loss = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    y_true, y_pred = [], []
    
    # Evaluate data for one epoch
    for batch in dataloader:
        # Step 1: Load batch data onto GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Step 2: Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Step 3: Perform a forward pass
            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)
            logits = outputs[0]
        # Step 4: Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Step 5: Calculate the accuracy and loss for this batch
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        eval_loss += loss_fn(outputs[0], b_labels).item()
        nb_eval_steps += 1
        
        y_true.extend(label_ids)
        y_pred.extend(np.argmax(logits, axis=1))

    # Calculate the average accuracy and loss over all of the batches
    avg_eval_accuracy = eval_accuracy / nb_eval_steps
    avg_eval_loss = eval_loss / len(dataloader)
    
    # Calculate classification metrics
    acc = accuracy_score(y_true, y_pred)
    clf_report = classification_report(y_true, y_pred, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)

    return {
        'Accuracy': acc,
        'F1 Score (Negative)': clf_report['0']['f1-score'],
        'F1 Score (Positive)': clf_report['1']['f1-score'],
        'Precision (Negative)': clf_report['0']['precision'],
        'Precision (Positive)': clf_report['1']['precision'],
        'Recall (Negative)': clf_report['0']['recall'],
        'Recall (Positive)': clf_report['1']['recall'],
        'Confusion Matrix': cm
    }

#model.load_state_dict(torch.load('best_model_state.bin'))
evaluate_bert_model(model, validation_dataloader)

{'Accuracy': 0.920125,
 'F1 Score (Negative)': 0.9195922989807475,
 'F1 Score (Positive)': 0.9206506891841548,
 'Precision (Negative)': 0.926094890510949,
 'Precision (Positive)': 0.9143153117600631,
 'Recall (Negative)': 0.9131803868645973,
 'Recall (Positive)': 0.9270744760666233,
 'Confusion Matrix': array([[18270,  1737],
        [ 1458, 18535]])}

In [14]:
# Defining a function to make predictions on new text
def predict(text):
    # Load the saved model state
    model.load_state_dict(torch.load('best_model_state.bin'))
    # Put the model in evaluation mode
    model.eval()
    # Tokenize the text and convert to input features
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    # Move the inputs to the device the model is on
    inputs.to(device)
    # Make the prediction
    with torch.no_grad():
        outputs = model(inputs['input_ids'], inputs['attention_mask'])
    logits = outputs[0]
    # Get the predicted class
    predicted_class = torch.argmax(logits).item()
    return predicted_class